In [1]:
# -*- coding:utf-8 -*-
import pandas as pd
import numpy as np
import sys,datetime,calendar,requests

In [2]:
# import Custom module
import OptionPricing as OP
sys.path.append('../')
# import OF2,OF1,OF4
import APICenter as API
import DataCenter as DATA
Api = API.API()
TestModel = True
if TestModel:
    Data = DATA.TestDataCenter()
else:
    Data = DATA.DataCenter()

In [4]:
def getrealtimeDataViaAPI():
    return Data.LoadOptionRealTimeDataViaAPI()
def get50ETFrealtimeDataViaAPI():
    return Data.Load50ETFRealTimeDataViaAPI()

In [55]:
class ServiceAPIforBackUp():

    def __init__(self):

        pass

    def checkNonClosedDay(self, date, apiurl='http://m.dsjcj.cc/tornado/common?a=check_holiday&date='):
        """ http://m.dsjcj.cc/tornado/common?a=check_holiday&date=2018-4-25

            用这个接口判断是否节假日，取data的值，如果为0为不休市，如果为1则为休市
            不休市返回：{"data": 0, "errmsg": "", "error": 0}
            休市返回：{"data": 1, "errmsg": "", "error": 0}"""
        response = requests.get(apiurl + date)
        if response.status_code == 200:
            import ast
            return int(ast.literal_eval(response.text)['data'])
        else:
            raise ValueError('info: response code :%d' %
                             (response.status_code))
        
    """def checkNonClosedDay(self, date, api=Api.ServiceAPI):
        # use api to check whether closed day
         http://m.dsjcj.cc/tornado/common?a=check_holiday&date=2018-4-25
        用这个接口判断是否节假日，取data的值，如果为0为不休市，如果为1则为休市
        不休市返回：{"data": 0, "errmsg": "", "error": 0}
        休市返回：{"data": 1, "errmsg": "", "error": 0}
        # 不休市 True;休市 False
        return True if api.checkNonClosedDay(date) == 0 else False"""

    def NextNonClosedDay(self, date):
        
        cNCD= True if self.checkNonClosedDay(date) == 0 else False
        # date %Y-%M-%D: '2018-04-25'
        if cNCD:#self.checkNonClosedDay(date):
            return date
        else:
            date = datetime.datetime.strptime(date, "%Y-%m-%d") + datetime.timedelta(days=1)
            return self.NextNonClosedDay(date.strftime('%Y-%m-%d'))

    def get4thWedandnextNonClosedDay(self, n=datetime.datetime.now()):
        year, month = n.year, n.month
        import calendar
        # 期权合约的到期日为到期月份的第四个星期三，该日为国家法定节假日、本所休市日的，顺延至下一个交易日
        s = \
        pd.DataFrame(calendar.monthcalendar(year, month), columns=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])[
            'Wed'].values
        if s[0] == 0:
            date = '%d-%s-%d' % (year, str(month) if str(month).startswith('1') else '0' + str(month), s[4])
        else:
            date = '%d-%s-%d' % (year, str(month) if str(month).startswith('1') else '0' + str(month), s[3])
        return self.NextNonClosedDay(date)  # 期权合约的到期日为到期月份的第四个星期三，该日为国家法定节假日、本所休市日的，顺延至下一个交易日

def Str2Datetime(testdatestr,formats='%Y-%m-%d'):
    return datetime.datetime.strptime(testdatestr,formats)
def YM2DatetimeStr(YM,century=21):
    Y=str(int(YM))[:2]
    M=str(int(YM))[2:]
    return '%d'%(century-1)+Y+'-'+M+'-01'

In [7]:
#s:Underlying Assets Price
#k:Strike Price
#r:risk free interest rate
#T:avaiable time
#sigma:square root of annual variance
#cp: call or put fee
#g: dividend yield
# dividends:continuous
op=OP.OptionPricing()
s,k,r,T,sigma,cp,g=2.654,2.45,0.03,1.0/12,0.2934,0.22,0
dividends='continuous'
#回望型波动率
op.BSPricing(s,k,r,T,sigma,cp,g,dividends=dividends)

0.03110753889407275

In [6]:
# get real time info:k and s
df=getrealtimeDataViaAPI()
df50ETF=get50ETFrealtimeDataViaAPI()
df50ETF

u'var hq_str_sh510050="50ETF,2.639,2.646,2.654,2.666,2.617,2.653,2.654,499165604,1319272213.000,381951,2.653,624300,2.652,156200,2.651,1949700,2.650,513900,2.649,504700,2.654,1501000,2.655,559700,2.656,373700,2.657,183900,2.658,2018-05-03,15:00:00,00";\n'

In [68]:
#get time span info
testdf=df.head(1)
YM=testdf['YM'].values[0]
dd=ServiceAPIforBackUp().get4thWedandnextNonClosedDay(Str2Datetime(YM2DatetimeStr(YM),'%Y-%m-%d'))
timespan=(Str2Datetime(dd)-datetime.datetime.today()).days
timespan>0

True

In [17]:
tef=df50ETF.split(';')[0].split('=')[1].strip('"').split(',')

In [18]:
tef

[u'50ETF',
 u'2.639',
 u'2.646',
 u'2.654',
 u'2.666',
 u'2.617',
 u'2.653',
 u'2.654',
 u'499165604',
 u'1319272213.000',
 u'381951',
 u'2.653',
 u'624300',
 u'2.652',
 u'156200',
 u'2.651',
 u'1949700',
 u'2.650',
 u'513900',
 u'2.649',
 u'504700',
 u'2.654',
 u'1501000',
 u'2.655',
 u'559700',
 u'2.656',
 u'373700',
 u'2.657',
 u'183900',
 u'2.658',
 u'2018-05-03',
 u'15:00:00',
 u'00']

'2018-06-27'